In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 1. Preparación de los Datos (sin CUDA)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

batch_size = 512
#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

#trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
# Cargar el conjunto de datos para entrenamiento
trainset = torchvision.datasets.ImageFolder(              #cargamos las imagenes de la carpeta train, cada carpeta dentro de train es una clase 
    root=r"../Datasets/train/",
    transform=transform             #Aplicamos las transformaciones anteriores
)

trainloader = DataLoader(trainset, batch_size=4, shuffle=True)      #cargamos imagenes en batches de 4 imagenes para optimizar y las organizamos aleatoriamente


# Cargar el conjunto de datos para prueba
testset = torchvision.datasets.ImageFolder(
    root=r"../Datasets/test/",
    transform=transform
)
testloader = DataLoader(testset, batch_size=5, shuffle=True)  # Batch size de 5 para visualizar

# 2. Modelo MLP Mejorado
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 2048)
        self.bn1 = nn.BatchNorm1d(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.bn2 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, 512)
        self.bn3 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(0.2)  # Reducimos Dropout a 0.2

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

# 3. Configuración del Modelo
modelo = MLP()
criterio = nn.CrossEntropyLoss()
optimizador = optim.Adam(modelo.parameters(), lr=0.002, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizador, step_size=10, gamma=0.5)

# 4. Función de Evaluación
def calcular_precision(loader):
    modelo.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imagenes, etiquetas in loader:
            outputs = modelo(imagenes)
            _, predicciones = torch.max(outputs, 1)
            correct += (predicciones == etiquetas).sum().item()
            total += etiquetas.size(0)
    return 100 * correct / total

# 5. Entrenamiento Optimizado
num_epochs = 50
for epoch in range(num_epochs):
    modelo.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        optimizador.zero_grad()
        outputs = modelo(inputs)
        loss = criterio(outputs, labels)
        loss.backward()
        optimizador.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    scheduler.step()
    val_precision = calcular_precision(testloader)

    print(f'Época {epoch + 1}, Pérdida: {running_loss / len(trainloader):.4f}, '
          f'Precisión de Validación: {val_precision:.2f}%')

# 6. Evaluación Final
test_precision = calcular_precision(testloader)
print(f"Precisión final en el conjunto de prueba: {test_precision:.2f}%")


Época 1, Pérdida: 1.9463, Precisión de Validación: 10.68%
Época 2, Pérdida: 1.8953, Precisión de Validación: 11.85%


KeyboardInterrupt: 